In [26]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20

# device will determine whether to run the training on GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [27]:
# Use transforms.compose method to reformat images for modeling,
# and save to variable all_transforms for later use
all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          std=[0.2023, 0.1994, 0.2010])
                                     ])
# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = all_transforms,
                                             download = True)

# Create Testing dataset
test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                            train = False,
                                            transform = all_transforms,
                                            download=True)

# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

Files already downloaded and verified
Files already downloaded and verified


In [28]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [38]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        print(images.shape)
        print(labels.shape)
        break
    break

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [37]:
labels = []
for data in train_loader:
    _, batch_labels = data
    labels.extend(batch_labels.tolist())
    
labels_tensor = torch.tensor(labels)

unique_labels, label_counts = torch.unique(labels_tensor, return_counts=True)

print(f"unique_lables: {unique_labels}")
print(f"label_counts: {label_counts}")

unique_lables: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
label_counts: tensor([5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000])


In [41]:
model = ConvNeuralNet(num_classes)
model.to(device)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        print(loss)
        print(outputs)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        break #Test
    
    print(f'Epoch [{epoch + 1}/{num_epochs}, Loss: {"{:.4f}".format(loss.item())}]')
    break #Test

tensor(2.2986, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([[-4.4874e-02,  4.3811e-02, -7.8075e-02, -3.7412e-02,  5.8519e-02,
          3.2906e-02, -1.0355e-01,  5.7789e-02, -1.3436e-01, -8.9899e-02],
        [-1.3819e-02,  2.0082e-02, -7.8813e-02,  5.6201e-03,  6.8872e-02,
         -5.0184e-02, -9.7252e-02,  7.3458e-02, -1.1611e-01, -5.8291e-02],
        [-4.3640e-02,  4.7339e-02, -6.3329e-02, -1.3951e-02,  4.7281e-02,
         -1.3054e-02, -7.2078e-02,  4.2382e-02, -1.6601e-01, -5.8376e-03],
        [-6.3164e-03,  1.9979e-02, -6.3245e-02, -1.4167e-02,  7.7432e-02,
          1.2768e-03, -8.9522e-02,  5.8453e-02, -1.0551e-01, -5.0332e-02],
        [-1.2298e-02, -9.7659e-03, -8.8244e-02, -1.4515e-02,  6.2635e-02,
         -2.0959e-02, -1.0211e-01,  5.3557e-02, -1.2274e-01, -5.4545e-02],
        [ 1.1532e-02,  2.1491e-02, -3.5479e-02,  1.0279e-02,  1.8643e-02,
         -3.0367e-02, -1.1701e-01,  1.0353e-01, -1.7915e-01, -7.8921e-02],
        [-1.5718e-02,  1.2292e-02, -5.4816e-02

In [31]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f'Accuracy of the network on the {50000} images: {100 * correct / total}%')

Accuracy of the network on the 50000 images: 82.798%
